<a href="https://colab.research.google.com/github/marielnr/Hyperbolic-Tree-Messier-Hipparcos-/blob/main/HyperbolicTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import kagglehub
import os
import json

In [25]:
path = kagglehub.dataset_download("konivat/hipparcos-star-catalog")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'hipparcos-star-catalog' dataset.
Path to dataset files: /kaggle/input/hipparcos-star-catalog


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
os.listdir(path)

['hipparcos-voidmain.csv']

In [28]:
df_messier = pd.read_csv("catalogue-de-messier.csv", sep=';')
df_hipparcos = pd.read_csv(f"{path}/hipparcos-voidmain.csv")

In [29]:
!head -n 10 catalogue-de-messier.csv


﻿Messier;NGC;Object type / Type d'objet;Season / Saison;Magnitude;Constellation (EN);Constellation (FR);Constellation (Latin);RA (Right Ascension);Dec (Declinaison);Distance (l.y / a. l.);Size / Dimensions;Discoverer / Découvreur;Year / Année;Image;URL de l'image;Constellation
M34;NGC 1039;Open Cluster / Amas Ouvert;Autumn / Automne;5;Perseus;Persée;Perseus;02:42:07.40;+42:44:46.1;1450;35,0';Messier;1764;http://www.lasam.ca/messier/M034.JPG;https://www.datastro.eu/api/explore/v2.1/catalog/datasets/catalogue-de-messier/files/f77873064c67256350aa5817915fe480;Per
M13;NGC 6205;Globular Cluster / Amas Globulaire;Summer / Été;5;Hercules;Hercule;Hercules;16:41:41.63;+36:27:40.7;23500;23,2';Halley;1714;http://www.lasam.ca/messier/M013.JPG;https://www.datastro.eu/api/explore/v2.1/catalog/datasets/catalogue-de-messier/files/50043671892eb3ae293cac3cb5fd0cab;Her
M26;NGC 6694;Open Cluster / Amas Ouvert;Summer / Été;8;Shield;L’Écu de Sobieski;Scutum;18:45:18.66;-09:23:01.0;5100;15,0';Le Gentil;1749;

#Estructura Jerárquica Deseada
Raíz → Constelación → Tipo de Objeto Messier → Objeto Messier (con URL de imagen) → Estrellas Hipparcos Cercanas

In [30]:
df_messier["Constellation"]

,Constellation
0,Per
1,Her
2,Sct
3,Ori
4,NaN
...,...
105,And
106,Leo
107,CVn
108,Cyg


In [31]:
print(df_messier.columns)

Index(['Messier', 'NGC', 'Object type / Type d'objet', 'Season / Saison',
       'Magnitude', 'Constellation (EN)', 'Constellation (FR)',
       'Constellation (Latin)', 'RA (Right Ascension)', 'Dec (Declinaison)',
       'Distance (l.y / a. l.)', 'Size / Dimensions',
       'Discoverer / Découvreur', 'Year / Année', 'Image', 'URL de l'image',
       'Constellation'],
      dtype='object')


In [32]:
df_messier.head()

,Messier,NGC,Object type / Type d'objet,Season / Saison,Magnitude,Constellation (EN),Constellation (FR),Constellation (Latin),RA (Right Ascension),Dec (Declinaison),Distance (l.y / a. l.),Size / Dimensions,Discoverer / Découvreur,Year / Année,Image,URL de l'image,Constellation
0,M34,NGC 1039,Open Cluster / Amas Ouvert,Autumn / Automne,5,Perseus,Persée,Perseus,02:42:07.40,+42:44:46.1,1450.0,"35,0'",Messier,1764.0,http://www.lasam.ca/messier/M034.JPG,https://www.datastro.eu/api/explore/v2.1/catal...,Per
1,M13,NGC 6205,Globular Cluster / Amas Globulaire,Summer / Été,5,Hercules,Hercule,Hercules,16:41:41.63,+36:27:40.7,23500.0,"23,2'",Halley,1714.0,http://www.lasam.ca/messier/M013.JPG,https://www.datastro.eu/api/explore/v2.1/catal...,Her
2,M26,NGC 6694,Open Cluster / Amas Ouvert,Summer / Été,8,Shield,L’Écu de Sobieski,Scutum,18:45:18.66,-09:23:01.0,5100.0,"15,0'",Le Gentil,1749.0,http://www.lasam.ca/messier/M026.JPG,https://www.datastro.eu/api/explore/v2.1/catal...,Sct
3,M43,NGC 1982,Reflection Nebula / Nébuleuse à réflexion,Winter / Hiver,9,"Orion, Hunter",Orion,Orion,05:35:31.38,-05:16:02.9,1500.0,"20,0' x 15,0'",Mairan,1750.0,http://www.lasam.ca/messier/M043.JPG,https://www.datastro.eu/api/explore/v2.1/catal...,Ori
4,M45,NaN,Open Cluster / Amas Ouvert,Winter / Hiver,1,NaN,NaN,NaN,NaN,NaN,410.0,"120,0'",NaN,NaN,http://www.lasam.ca/messier/M045.JPG,https://www.datastro.eu/api/explore/v2.1/catal...,NaN


In [33]:
df_messier = df_messier.drop(columns=['Constellation','Constellation (FR)','Constellation (Latin)'])
df_messier=df_messier.rename(columns={'Constellation (EN)':'Constellation'})

In [34]:
column_renames = {
    # El error crítico: renombrar el tipo de objeto
    'Object type / Type d\'objet': 'ObjectType',

    # Renombrar las coordenadas para simplificar
    'RA (Right Ascension)': 'RA',
    'Dec (Declinaison)': 'Dec',

    # Renombrar distancia y URL
    'Distance (l.y / a. l.)': 'Distance_ly',
    'URL de l\'image': 'ImageURL',

    # Si existiera 'Constellation (Latin)', también deberías renombrarla,
    # pero como solo tienes 'Constellation', asumiremos que ya está bien.
}

In [35]:
df_messier=df_messier.rename(columns=column_renames)

In [36]:
def replace_nan(obj):
    if isinstance(obj, float) and np.isnan(obj):
        return None
    return obj

In [37]:
import pandas as pd
import json
# import numpy as np # Necesario si sus NaN son de numpy

# 1. FUNCIÓN HELPER PARA MANEJAR NaN/NULLs EN JSON
# Reemplaza los NaN con None, que se serializa a 'null' en JSON.
def replace_nan(obj):
    if pd.isna(obj):
        return None
    raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

def build_messier_tree_hierarchical(df_messier):
    """
    Construye una jerarquía anidada: Raíz -> Constelación -> Objeto Messier.

    Args:
        df_messier (pd.DataFrame): DataFrame con datos del catálogo.

    Returns:
        dict: Estructura de árbol anidada para D3-Hypertree.
    """

    # --- PASO CRÍTICO 1: Limpieza de Datos (Eliminar filas con cualquier NaN/null) ---
    # Esto borra cualquier fila que tenga un valor faltante en CUALQUIER columna.
    df_cleaned = df_messier.dropna(how='any').copy()

    # Asegurarse de que la columna 'Constellation' sea de tipo string para agrupar
    df_cleaned['Constellation'] = df_cleaned['Constellation'].astype(str)

    root_id = "catalogs_root"
    total_leafs = len(df_cleaned)

    # Nodo Raíz
    tree_data = {
        "name": "Universo Observable / Catálogos Messier & Hipparcos",
        "numLeafs": total_leafs,
        "id": root_id,
        "children": []
    }

    # Agrupar por Constelación
    constellation_groups = df_cleaned.groupby('Constellation')

    # --- PASO CRÍTICO 2: Creación de la Jerarquía Anidada (Constelación como rama) ---
    for constel_name, group in constellation_groups:
        # Nodo Intermedio: La Constelación
        constel_node = {
            "name": constel_name,
            "numLeafs": len(group), # Cantidad de hojas (objetos) en esta rama
            "id": f"constel_{constel_name.replace(' ', '_')}",
            "children": []
        }

        # Nodos Hojas: Los Objetos Messier
        for index, row in group.iterrows():
            messier_name = f"{row['Messier']} / {constel_name}"

            messier_node = {
                "name": messier_name,
                "numLeafs": 0, # Las hojas siempre tienen 0 numLeafs
                "id": row['Messier'],

                # Agregar las propiedades como metadatos
                "ObjectType": row['ObjectType'],
                "Magnitude": row['Magnitude'],
                "Distance_ly": row['Distance_ly'],
                "ImageURL": row['ImageURL'],
                "RA": row['RA'],
                "Dec": row['Dec'],

                "children": [] # Nodo terminal (Hoja)
            }
            constel_node["children"].append(messier_node)

        tree_data["children"].append(constel_node)

    return tree_data

# 2. PROCESAR DATOS Y GENERAR JSON
final_tree = build_messier_tree_hierarchical(df_messier)

# --- PASO CRÍTICO 3: Guardar el JSON en formato de ARRAY para D3-Hypertree ---
# Esto corrige el "TypeError: json.forEach is not a function"
with open('hyperbolic_tree_data_hierarchical.json', 'w') as f:
    json.dump(final_tree, f, indent=4, default=replace_nan)


In [38]:
df_messier['Constellation'].unique()

array(['Perseus', 'Hercules', 'Shield', 'Orion, Hunter', nan, 'Virgin',
       'Lion', 'Archer', 'Triangle', 'Serpent Holder', 'Crab',
       'Hair of Berenice', 'Unicorn', 'Big Bear', 'Andromeda', 'Bull',
       'Goat', 'Cassiopeia', 'Stern,Poop deck', 'Arrow', 'Hound Dogs',
       'Water Bearer', 'Scorpion', 'Charioteer', 'Hydra, Sea Snake',
       'Hare', 'Great Dog', 'Little Fox', 'Lyre', 'Crow', 'Winged Horse',
       'Fishes', 'Twins', 'Sea-monster', 'Swan'], dtype=object)

In [39]:
df_hipparcos.columns

Index(['Catalog', 'HIP', 'Proxy', 'RAhms', 'DEdms', 'Vmag', 'VarFlag',
       'r_Vmag', 'RAdeg', 'DEdeg', 'AstroRef', 'Plx', 'pmRA', 'pmDE',
       'e_RAdeg', 'e_DEdeg', 'e_Plx', 'e_pmRA', 'e_pmDE', 'DE:RA', 'Plx:RA',
       'Plx:DE', 'pmRA:RA', 'pmRA:DE', 'pmRA:Plx', 'pmDE:RA', 'pmDE:DE',
       'pmDE:Plx', 'pmDE:pmRA', 'F1', 'F2', '---', 'BTmag', 'e_BTmag', 'VTmag',
       'e_VTmag', 'm_BTmag', 'B-V', 'e_B-V', 'r_B-V', 'V-I', 'e_V-I', 'r_V-I',
       'CombMag', 'Hpmag', 'e_Hpmag', 'Hpscat', 'o_Hpmag', 'm_Hpmag', 'Hpmax',
       'HPmin', 'Period', 'HvarType', 'moreVar', 'morePhoto', 'CCDM', 'n_CCDM',
       'Nsys', 'Ncomp', 'MultFlag', 'Source', 'Qual', 'm_HIP', 'theta', 'rho',
       'e_rho', 'dHp', 'e_dHp', 'Survey', 'Chart', 'Notes', 'HD', 'BD', 'CoD',
       'CPD', '(V-I)red', 'SpType', 'r_SpType'],
      dtype='object')

In [40]:
def replace_nan(obj):
    if isinstance(obj, float) and np.isnan(obj):
        return None
    return obj

# --- 2. FUNCIÓN PRINCIPAL DE GENERACIÓN DE ETIQUETAS ---
def generate_labels_json(df_messier, df_hipparcos):
    """Genera el archivo de mapeo de etiquetas (labels.json) para Messier e Hipparcos."""

    labels_data = {}

    # Etiqueta para el nodo raíz, si alguna vez la necesitas:
    labels_data["catalogs_root"] = "Universo Observable"

    # Etiquetar Objetos Messier (M#)
    for index, row in df_messier.iterrows():
        messier_id = row['Messier']

        # Obtenemos la Constelación de forma segura
        constel_val = None
        if 'Constellation' in row and not pd.isna(row['Constellation']):
            constel_val = row['Constellation']

        # Formato de etiqueta: "M# (Constelación)"
        friendly_name = f"{messier_id} ({constel_val})" if constel_val else messier_id

        # Mapeamos ID -> Nombre legible
        labels_data[messier_id] = friendly_name

    # Etiquetar Estrellas Hipparcos (HIP XXXX)
    if df_hipparcos is not None and 'HIP' in df_hipparcos.columns:

        # Usamos 'HD' para el nombre de la estrella, si existe en tu DF
        hipparcos_id_col = 'HIP'
        hipparcos_name_col = 'HD'

        for index, row in df_hipparcos.iterrows():
            # El ID que usaste en el árbol: "HIP 12345"
            star_id = f"HIP {row[hipparcos_id_col]}"

            star_name = None

            # Intentamos usar el nombre HD como etiqueta legible
            if hipparcos_name_col in row and not pd.isna(row[hipparcos_name_col]):
                # Formateamos como "HD XXXXX" (convertimos a int para limpiar el float)
                star_name = f"HD {int(row[hipparcos_name_col])}"

            # El valor de la etiqueta es el nombre HD o el ID HIP
            labels_data[star_id] = star_name if star_name else star_id

    return labels_data

# --- 3. EXPORTACIÓN DEL ARCHIVO ---

# Asegúrate de que df_messier y df_hipparcos están cargados antes de este punto
# if 'df_messier' not in locals() or 'df_hipparcos' not in locals():
#     print("ERROR: Asegúrate de cargar df_messier y df_hipparcos primero.")
# else:
labels_map = generate_labels_json(df_messier, df_hipparcos)

with open('labels.json', 'w') as f:
    # Usamos replace_nan para la limpieza final
    json.dump(labels_map, f, indent=4, default=replace_nan)

print("¡Archivo labels.json generado con éxito!")

¡Archivo labels.json generado con éxito!


In [41]:
print(df_hipparcos.head())

  Catalog  HIP Proxy        RAhms        DEdms  Vmag  VarFlag r_Vmag  \
0       H    1   NaN  00 00 00.22  +01 05 20.4  9.10      NaN      H   
1       H    2   NaN  00 00 00.91  -19 29 55.8  9.27      NaN      G   
2       H    3   NaN  00 00 01.20  +38 51 33.4  6.61      NaN      G   
3       H    4   NaN  00 00 02.01  -51 53 36.8  8.06      NaN      H   
4       H    5   NaN  00 00 02.39  -40 35 28.4  8.55      NaN      H   

      RAdeg      DEdeg  ... Survey  Chart  Notes        HD         BD  \
0  0.000912   1.089013  ...      S    NaN    NaN  224700.0  B+00 5077   
1  0.003797 -19.498837  ...    NaN    NaN    NaN  224690.0  B-20 6688   
2  0.005008  38.859286  ...      S    NaN    NaN  224699.0  B+38 5108   
3  0.008382 -51.893546  ...      S    NaN    NaN  224707.0        NaN   
4  0.009965 -40.591224  ...    NaN    NaN    NaN  224705.0        NaN   

          CoD         CPD  (V-I)red  SpType  r_SpType  
0         NaN         NaN      0.66      F5         S  
1         NaN   

In [42]:
df_hipparcos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118218 entries, 0 to 118217
Data columns (total 78 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Catalog    118218 non-null  object 
 1   HIP        118218 non-null  int64  
 2   Proxy      10925 non-null   object 
 3   RAhms      118218 non-null  object 
 4   DEdms      118218 non-null  object 
 5   Vmag       118217 non-null  float64
 6   VarFlag    11562 non-null   float64
 7   r_Vmag     118217 non-null  object 
 8   RAdeg      117955 non-null  float64
 9   DEdeg      117955 non-null  float64
 10  AstroRef   13734 non-null   object 
 11  Plx        117955 non-null  float64
 12  pmRA       117955 non-null  float64
 13  pmDE       117955 non-null  float64
 14  e_RAdeg    117955 non-null  float64
 15  e_DEdeg    117955 non-null  float64
 16  e_Plx      117955 non-null  float64
 17  e_pmRA     117955 non-null  float64
 18  e_pmDE     117955 non-null  float64
 19  DE:RA      117955 non-n